In [ ]:
%load_ext autoreload
%autoreload 2

from dataset_utils import adni_loader
import json
import os
import sys

import tensorflow as tf
print(tf.__version__)
import keras.backend as K
gpu_ids = [3]
# set gpu id and tf settings
os.environ['CUDA_VISIBLE_DEVICES']=','.join([str(g) for g in gpu_ids])
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
K.tensorflow_backend.set_session(tf.Session(config=config))

load_params_from_exp = './experiments/FewShotSeg_adni-unnorm-masked_100ul_subj-l_ims160-192-1_arch32-32-64-64-128-128_augflow-amp200_blur12_randmult0.5_flowaug-gen'

data_params_file = os.path.join(load_params_from_exp, 'data_params.json')
with open(data_params_file, 'r') as f:
    data_params = json.load(f)
data_params['vte_epoch'] = 500
arch_params_file = os.path.join(load_params_from_exp, 'arch_params.json')
with open(arch_params_file, 'r') as f:
    arch_params = json.load(f)

data_params['scale'] = 1  # always load full-scale data for evaluation
data_params['n_unlabeled'] = 1  # just load one for training, we only want the validation set
data_params['load_vols'] = False
ds = adni_loader.ADNIDataset(data_params)
_ = ds.load_dataset(debug=False)
vol_shape = (160, 192, 224, 1)



In [ ]:
use_glt_model = False

if use_glt_model:
    load_data_from_exp = './experiments/FewShotSeg_adni_all-ul_atlas-l_ims160-192-1_arch32-32-64-64-128-128_augflow-amp150_blur10_flowaug-gen'
    load_data_from_exp = './experiments/FewShotSeg_adni-unnorm-masked_100ul_atlas-l_ims160-192-1_arch32-32-64-64-128-128_vteaug-gen-halfres_wrapper_unets_seq_train_wrapper-e1000'
    load_data_from_exp = './experiments/GLT_adni-unnorm-masked_100ul_atlas-l-to-subjs_ims160-192-224-1_voxelmorph_flow_color_unet16-16-32-32-32-32_dec64-64-32-32-32-32_lr0.0005_cc_grad_l2_regfwt1_cc_wt1_win9_grad-si-l2-predgrad_regcwt0.5_l2_sigI0.1'
    #load_data_from_exp = './experiments/GLT_adni-unnorm-masked_100ul_atlas-l-to-subjs_ims160-192-224-1_voxelmorph_flow_color_unet16-16-32-32-32-32_dec64-64-32-32-32-32_lr0.0005_cc_vm_grad_l2_regfwt1_cc_vm_wt1_win9_grad-si-l2-predgrad_regcwt0.5_l2_sigI0.1'
    #load_data_from_exp = './experiments/GLT_adni-unnorm-masked_100ul_atlas-l-to-subjs_ims160-192-224-1_voxelmorph2_flow_color_unet16-32-32-32_dec64-64-32-32-32_lr0.0005_cc_grad_l2_regfwt1_cc_wt1_win9_grad-si-l2-predgrad_regcwt0.5_l2_sigI0.1'
    load_data_from_exp = './experiments/GLT_adni-unnorm-masked_100ul_atlas-l-to-subjs_ims160-192-224-1_voxelmorph2_flow_color_unet16-32-32-32_dec64-64-32-32-32_lr0.0005_cc_vm_grad_l2_regfwt1_cc_vm_wt1_win9_grad-si-l2-predgrad_regcwt0.5_l2_sigI0.1'
    load_data_from_exp = './experiments/GLT_adni-sf0.5-unnorm-masked_100ul_atlas-l-to-subjs_ims160-192-224-1_voxelmorph2_flow_color_unet16-32-32-32_dec64-64-32-32-32_lr0.0005_cc_vm_grad_l2_regfwt1_cc_vm_wt1_win5_grad-si-l2-predgrad_regcwt0.5_l2_sigI0.1'
    
    
    data_params_file = os.path.join(load_data_from_exp, 'data_params.json')
    with open(data_params_file, 'r') as f:
        data_params = json.load(f)
    data_params['vte_epoch'] = 500
    arch_params_file = os.path.join(load_data_from_exp, 'arch_params.json')
    with open(arch_params_file, 'r') as f:
        arch_params = json.load(f)

    sys.path.append('../cnn_utils')
    import file_utils, vis_utils

    # load segmenter because it has the half-res wrapper logic
    from experiments_VTE import FewShotSegmentationExperimentClass, GLTExperimentClass
    #exp = FewShotSegmentationExperimentClass.ExperimentSegmenter(data_params, arch_params)
    exp = GLTExperimentClass.ExperimentGlobalLocalTransforms(data_params, arch_params)

    model_name, exp_dir, figures_dir, logs_dir, models_dir = file_utils.make_output_dirs(exp.get_model_name(), exp_root='./experiments/', prompt_delete=False)
    exp.save_exp_info(exp_dir, figures_dir, models_dir, logs_dir)

    exp.load_data(load_fewer=True)

In [ ]:
from keras.models import load_model
from networks import transform_network_utils

if not use_glt_model:
    sys.path.append('../voxelmorph')
    sys.path.append('../voxelmorph/src')
    import src.networks as vm_networks
    from dense_3D_spatial_transformer import Dense3DSpatialTransformer

    voxelmorph_model = load_model(
    #     #'/afs/csail.mit.edu/u/x/xamyzhao/voxelmorph/models/vm2_l2.h5',
    #     '/afs/csail.mit.edu/u/x/xamyzhao/LPAT/experiments/voxelmorph/vm2_cc_AtoS_iter54000.h5',
        '/afs/csail.mit.edu/u/x/xamyzhao/LPAT/experiments/voxelmorph/vm2_cc_AtoUMS_iter15000.h5',
    #     #'/afs/csail.mit.edu/u/x/xamyzhao/LPAT/experiments/voxelmorph/vm2_l2_StoS_ftiter9500.h5',
        custom_objects={'Dense3DSpatialTransformer': Dense3DSpatialTransformer},
        compile=False,
    )
    voxelmorph_model.summary()
elif 'sf0.5' in exp.model_name:
    exp.create_models()
    exp.load_models(models_dir, 500)
    voxelmorph_model = transform_network_utils.vm_halfres_wrapper(
        full_img_shape=(160, 192, 224, 1),
        model=exp.unet_flow,
        upsample_outputs=[True, False],
        scale_outputs=[2, 1],
    )
elif 'GLT' in exp.model_name:
    exp.create_models()
    exp.load_models(models_dir, 'latest')
    voxelmorph_model = exp.unet_flow
    
vol_warp_model = transform_network_utils.warp_model(
    img_shape=vol_shape,
    interp_mode='linear'
)

seg_warp_model = transform_network_utils.warp_model(
    img_shape=vol_shape[:-1] + (1,),
    interp_mode='nearest'
)


In [ ]:
from keras.layers import Conv2D, Conv3D
for l in voxelmorph_model.layers:
    
    if isinstance(l, Conv3D):
        print(l.get_weights()[0].shape)

In [ ]:
import numpy as np
import medipy_metrics
import classification_utils
import IPython
import PIL
import vte_runner
import vis_utils
label_mapping = vte_runner.voxelmorph_labels
# load each subject, then evaluate each slice
n_test_examples = len(ds.files_labeled_valid)
n_test_examples = 50
dice_per_label = np.zeros((len(label_mapping,)))

source_X = ds.X_atlas
source_Y = ds.Y_atlas

# source_X, source_Y =  adni_loader._load_vol_and_seg(ds.files_labeled_train[0])
# source_X = source_X[np.newaxis]
# source_Y = source_Y[np.newaxis]

for bi in range(n_test_examples):
    print('Evaluating test subject {} of {}'.format(bi, n_test_examples))
    target_X, target_Y =  adni_loader._load_vol_and_seg(ds.files_labeled_valid[bi], mask_vol = ds.params['masked'])
    n_slices = target_X.shape[2]
    preds = np.zeros(target_Y.shape)

    preds = voxelmorph_model.predict([source_X, target_X[np.newaxis]])

    if use_glt_model:
        # GLT rather than voxelmorph
        warped, warp = preds[-1], preds[0]
    else:
        warped, warp = preds
        warp = warp[..., [1, 0, 2]]  # since voxelmorph uses a different SpatialTransformer
    
    warped = vol_warp_model.predict([source_X, warp])
    #warped, warp = preds[0], preds[1]
    slice_idx = np.random.choice(source_X.shape[-2], 1)[0]
    IPython.display.display(PIL.Image.fromarray(
        vis_utils.label_ims(
            np.concatenate([source_X[:, :, :, slice_idx] , target_X[np.newaxis, :, :, slice_idx], warped[:, :, :, slice_idx]], axis=0),
            concat_axis=1
        )
    ))
    
    pred = seg_warp_model.predict([source_Y[..., np.newaxis], warp])
    
    subject_dice_per_label = medipy_metrics.dice(target_Y[np.newaxis], pred, labels=label_mapping)[0, :]
    dice_per_label += subject_dice_per_label
    print(subject_dice_per_label)
dice_per_label /= float(n_test_examples)





In [ ]:
print('Mean dice: {}'.format(np.mean(dice_per_label[1:])))
print('Dice per label: {}'.format(dice_per_label))
for li, l in enumerate(ds.label_mapping):
    print('{}: {}'.format(l, dice_per_label[li]))



In [ ]:
# # now try subject-to-atlas just once
# source_X, source_Y =  adni_loader._load_vol_and_seg(ds.files_labeled_valid[bi])

# target_X, target_Y =  ds.X_atlas, ds.Y_atlas
# n_slices = target_X.shape[2]
# preds = np.zeros(target_Y.shape)

# print(source_X.shape)
# print(target_X.shape)
# print(source_Y.shape)
# warped, warp = voxelmorph_model.predict([source_X[np.newaxis], target_X])
# pred = seg_warp_model.predict([source_Y[np.newaxis,..., np.newaxis], warp])

# subject_dice_per_label = medipy_metrics.dice(target_Y[np.newaxis], pred, labels=label_mapping)[0, :]

# print(subject_dice_per_label)
# print(np.mean(subject_dice_per_label[1:]))

